 # Análise Exploratória de Dado
------------------------------------------------------
Este notebook realiza a análise exploratória dos dados da camada Gold.

Python Version: 3.13.2
Database: SQLite (gold_layer.db)

# Importação de Bibliotecas

In [1]:
import os
import sqlite3
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import calendar

# Configuração de estilo dos gráficos

In [12]:
plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = (14, 8)
plt.rcParams['font.size'] = 12
%matplotlib inline


# Conectar ao banco de dados da camada Gold

In [13]:
BASE_DIR = os.path.abspath(os.path.join(os.getcwd(), '..'))
GOLD_DB_PATH = os.path.join(BASE_DIR, 'gold', 'gold_layer.db')

# Funções auxiliares para executar consultas SQL

In [14]:
def run_query(query, conn):
    """Executa uma consulta SQL e retorna os resultados como um DataFrame."""
    return pd.read_sql_query(query, conn)

def print_section(title):
    """Imprime um cabeçalho de seção com formatação."""
    print("\n" + "="*80)
    print(f" {title} ".center(80, "="))
    print("="*80 + "\n")

# Parte A: Volume de pedidos por mês e análise de sazonalidade

In [15]:
def run_query(query, conn):
    """Executa uma consulta SQL e retorna os resultados como um DataFrame."""
    return pd.read_sql_query(query, conn)

def print_section(title):
    """Imprime um cabeçalho de seção com formatação."""
    print("\n" + "="*80)
    print(f" {title} ".center(80, "="))
    print("="*80 + "\n")

def analise_volume_pedidos(conn):
    print_section("A) Volume de pedidos por mês e análise de sazonalidade")
    
    query = """
    SELECT 
        year, month, 
        num_orders, total_revenue,
        num_customers, avg_ticket
    FROM agg_sales_by_time
    ORDER BY year, month
    """
    
    vendas_mensais = run_query(query, conn)
    
    vendas_mensais['mes_nome'] = vendas_mensais['month'].apply(lambda x: calendar.month_name[x])
    vendas_mensais['ano_mes'] = vendas_mensais.apply(lambda x: f"{x['year']}-{x['month']:02d}", axis=1)
    
    print("Volume de pedidos por mês:")
    print(vendas_mensais[['year', 'month', 'mes_nome', 'num_orders', 'total_revenue']])
    
    plt.figure(figsize=(16, 8))
    plt.plot(vendas_mensais['ano_mes'], vendas_mensais['num_orders'], marker='o', linewidth=2)
    plt.title('Volume de Pedidos por Mês', fontsize=18)
    plt.xlabel('Mês', fontsize=14)
    plt.ylabel('Número de Pedidos', fontsize=14)
    plt.xticks(rotation=45)
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()
    
    # Visualizar a receita total ao longo do tempo
    plt.figure(figsize=(16, 8))
    plt.plot(vendas_mensais['ano_mes'], vendas_mensais['total_revenue'], marker='o', linewidth=2, color='green')
    plt.title('Receita Total por Mês', fontsize=18)
    plt.xlabel('Mês', fontsize=14)
    plt.ylabel('Receita Total (R$)', fontsize=14)
    plt.xticks(rotation=45)
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()
    
    sazonalidade = vendas_mensais.groupby('month').agg({
        'num_orders': 'mean',
        'total_revenue': 'mean'
    }).reset_index()
    
    sazonalidade['mes_nome'] = sazonalidade['month'].apply(lambda x: calendar.month_name[x])
    sazonalidade = sazonalidade.sort_values('month')
    
    print("\nAnálise de sazonalidade (média mensal):")
    print(sazonalidade[['month', 'mes_nome', 'num_orders', 'total_revenue']])
    
    plt.figure(figsize=(16, 8))
    plt.plot(sazonalidade['mes_nome'], sazonalidade['num_orders'], marker='o', linewidth=2)
    plt.title('Sazonalidade - Média de Pedidos por Mês', fontsize=18)
    plt.xlabel('Mês', fontsize=14)
    plt.ylabel('Média de Pedidos', fontsize=14)
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()
    
    cv = sazonalidade['num_orders'].std() / sazonalidade['num_orders'].mean()
    meses_pico = sazonalidade.nlargest(3, 'num_orders')['mes_nome'].tolist()
    
    print(f"\nCoeficiente de variação dos pedidos mensais: {cv:.2f}")
    print(f"Meses de pico: {', '.join(meses_pico)}")
    
    conclusao = "Existe sazonalidade nas vendas" if cv > 0.1 else "Não existe sazonalidade significativa nas vendas"
    print(f"\nConclusão: {conclusao}")
    
    return vendas_mensais

# Parte B: Distribuição do tempo de entrega dos pedidos

In [16]:
def analise_tempo_entrega(conn):
    print_section("B) Distribuição do tempo de entrega dos pedidos")
    
    query = """
    SELECT 
        julianday(order_delivered_customer_date) - julianday(order_purchase_timestamp) AS tempo_entrega_dias,
        order_id
    FROM fact_sales
    WHERE order_delivered_customer_date IS NOT NULL
      AND order_purchase_timestamp IS NOT NULL
    GROUP BY order_id
    """
    
    tempo_entrega = run_query(query, conn)
    
    print("Estatísticas do tempo de entrega (em dias):")
    print(f"Média: {tempo_entrega['tempo_entrega_dias'].mean():.2f} dias")
    print(f"Mediana: {tempo_entrega['tempo_entrega_dias'].median():.2f} dias")
    print(f"Mínimo: {tempo_entrega['tempo_entrega_dias'].min():.2f} dias")
    print(f"Máximo: {tempo_entrega['tempo_entrega_dias'].max():.2f} dias")
    print(f"Desvio Padrão: {tempo_entrega['tempo_entrega_dias'].std():.2f} dias")
    
    
    percentuais = [10, 25, 50, 75, 90, 95, 99]
    for p in percentuais:
        print(f"Percentual {p}: {tempo_entrega['tempo_entrega_dias'].quantile(p/100):.2f} dias")
    
    plt.figure(figsize=(16, 8))
    sns.histplot(tempo_entrega['tempo_entrega_dias'], bins=50, kde=True)
    plt.title('Distribuição do Tempo de Entrega', fontsize=18)
    plt.xlabel('Tempo de Entrega (dias)', fontsize=14)
    plt.ylabel('Frequência', fontsize=14)
    plt.axvline(tempo_entrega['tempo_entrega_dias'].mean(), color='red', linestyle='--', 
                label=f'Média: {tempo_entrega["tempo_entrega_dias"].mean():.2f} dias')
    plt.axvline(tempo_entrega['tempo_entrega_dias'].median(), color='green', linestyle='--', 
                label=f'Mediana: {tempo_entrega["tempo_entrega_dias"].median():.2f} dias')
    plt.legend()
    plt.tight_layout()
    plt.show()
    
    plt.figure(figsize=(16, 6))
    sns.boxplot(x=tempo_entrega['tempo_entrega_dias'])
    plt.title('Boxplot do Tempo de Entrega', fontsize=18)
    plt.xlabel('Tempo de Entrega (dias)', fontsize=14)
    plt.tight_layout()
    plt.show()
    
    bins = [0, 7, 14, 21, 30, float('inf')]
    labels = ['até 7 dias', '8-14 dias', '15-21 dias', '22-30 dias', 'mais de 30 dias']
    tempo_entrega['categoria'] = pd.cut(tempo_entrega['tempo_entrega_dias'], bins=bins, labels=labels)
    
    contagem_categorias = tempo_entrega['categoria'].value_counts().sort_index()
    
    print("\nDistribuição por categoria de tempo de entrega:")
    print(contagem_categorias)
    
    plt.figure(figsize=(16, 8))
    sns.countplot(y='categoria', data=tempo_entrega, order=labels)
    plt.title('Distribuição por Categoria de Tempo de Entrega', fontsize=18)
    plt.xlabel('Número de Pedidos', fontsize=14)
    plt.ylabel('Categoria', fontsize=14)
    plt.tight_layout()
    plt.show()
    
    return tempo_entrega

# Parte C: Relação entre o valor do frete e a distância de entrega

In [17]:
def analise_frete_distancia(conn):
    print_section("C) Relação entre o valor do frete e a distância de entrega")
    
    query = """
    SELECT 
        fs.order_id,
        fs.freight_value,
        c.customer_zip_code_prefix,
        c.customer_state,
        s.seller_zip_code_prefix,
        s.seller_state
    FROM fact_sales fs
    JOIN dim_customers c ON fs.customer_id = c.customer_id
    JOIN dim_sellers s ON fs.seller_id = s.seller_id
    """
    
    dados_frete = run_query(query, conn)
    
    dados_frete['mesmo_estado'] = dados_frete['customer_state'] == dados_frete['seller_state']
    
    frete_por_estado = dados_frete.groupby('mesmo_estado').agg({
        'freight_value': ['mean', 'median', 'std', 'count']
    }).reset_index()
    
    frete_por_estado.columns = ['mesmo_estado', 'frete_medio', 'frete_mediana', 'frete_std', 'quantidade']
    
    print("Relação entre frete e estado (mesmo estado vs. estados diferentes):")
    print(frete_por_estado)
    
    plt.figure(figsize=(14, 8))
    sns.barplot(x='mesmo_estado', y='frete_medio', data=frete_por_estado)
    plt.title('Valor Médio do Frete por Relação de Estado', fontsize=18)
    plt.xlabel('Mesmo Estado', fontsize=14)
    plt.ylabel('Valor Médio do Frete (R$)', fontsize=14)
    plt.xticks([0, 1], ['Estados Diferentes', 'Mesmo Estado'])
    plt.tight_layout()
    plt.show()
    
    frete_por_estado_detalhado = dados_frete.groupby('customer_state').agg({
        'freight_value': ['mean', 'count']
    }).reset_index()
    
    frete_por_estado_detalhado.columns = ['estado_cliente', 'frete_medio', 'quantidade']
    frete_por_estado_detalhado = frete_por_estado_detalhado.sort_values('frete_medio', ascending=False)
    
    print("\nValor médio do frete por estado do cliente:")
    print(frete_por_estado_detalhado)
    
    plt.figure(figsize=(16, 10))
    sns.barplot(x='frete_medio', y='estado_cliente', data=frete_por_estado_detalhado)
    plt.title('Valor Médio do Frete por Estado do Cliente', fontsize=18)
    plt.xlabel('Valor Médio do Frete (R$)', fontsize=14)
    plt.ylabel('Estado do Cliente', fontsize=14)
    plt.tight_layout()
    plt.show()
    
    query_preco = """
    SELECT 
        fs.order_id,
        fs.freight_value,
        fs.price,
        c.customer_state,
        s.seller_state
    FROM fact_sales fs
    JOIN dim_customers c ON fs.customer_id = c.customer_id
    JOIN dim_sellers s ON fs.seller_id = s.seller_id
    """
    
    dados_completos = run_query(query_preco, conn)
    dados_completos['mesmo_estado'] = dados_completos['customer_state'] == dados_completos['seller_state']
    dados_completos['percentual_frete'] = (dados_completos['freight_value'] / dados_completos['price']) * 100
    
    dados_filtrados = dados_completos[dados_completos['percentual_frete'] <= 100]
    
    plt.figure(figsize=(16, 8))
    plt.scatter(dados_filtrados['price'], dados_filtrados['freight_value'], alpha=0.1)
    plt.title('Relação entre Preço do Produto e Valor do Frete', fontsize=18)
    plt.xlabel('Preço do Produto (R$)', fontsize=14)
    plt.ylabel('Valor do Frete (R$)', fontsize=14)
    plt.tight_layout()
    plt.show()
    
    plt.figure(figsize=(16, 8))
    sns.boxplot(x='mesmo_estado', y='percentual_frete', data=dados_filtrados)
    plt.title('Percentual do Frete em Relação ao Preço do Produto', fontsize=18)
    plt.xlabel('Mesmo Estado', fontsize=14)
    plt.ylabel('Percentual do Frete (%)', fontsize=14)
    plt.xticks([0, 1], ['Estados Diferentes', 'Mesmo Estado'])
    plt.tight_layout()
    plt.show()
    
    return dados_frete

# Parte D: Categorias de produtos mais vendidas em termos de faturamento

In [18]:
def analise_categorias_produtos(conn):
    print_section("D) Categorias de produtos mais vendidas em termos de faturamento")
    
    query = """
    SELECT 
        category,
        total_revenue,
        num_orders,
        avg_ticket
    FROM agg_sales_by_category
    ORDER BY total_revenue DESC
    """
    
    categorias = run_query(query, conn)
    
    print("Top 20 categorias de produtos por faturamento:")
    print(categorias.head(20))
    
    plt.figure(figsize=(16, 10))
    sns.barplot(x='total_revenue', y='category', data=categorias.head(10), palette='viridis')
    plt.title('Top 10 Categorias de Produtos por Faturamento', fontsize=18)
    plt.xlabel('Faturamento Total (R$)', fontsize=14)
    plt.ylabel('Categoria', fontsize=14)
    plt.tight_layout()
    plt.show()
    
    top10 = categorias.head(10).copy()
    plt.figure(figsize=(16, 10))
    sns.barplot(x='num_orders', y='category', data=top10, palette='viridis')
    plt.title('Número de Pedidos por Categoria (Top 10 em Faturamento)', fontsize=18)
    plt.xlabel('Número de Pedidos', fontsize=14)
    plt.ylabel('Categoria', fontsize=14)
    plt.tight_layout()
    plt.show()
    
    plt.figure(figsize=(16, 10))
    sns.barplot(x='avg_ticket', y='category', data=top10, palette='viridis')
    plt.title('Ticket Médio por Categoria (Top 10 em Faturamento)', fontsize=18)
    plt.xlabel('Ticket Médio (R$)', fontsize=14)
    plt.ylabel('Categoria', fontsize=14)
    plt.tight_layout()
    plt.show()
    
    total_faturamento = categorias['total_revenue'].sum()
    categorias['percentual_faturamento'] = (categorias['total_revenue'] / total_faturamento) * 100
    
    print("\nDistribuição percentual do faturamento por categoria:")
    print(categorias[['category', 'total_revenue', 'percentual_faturamento']].head(20))
    
    categorias_sorted = categorias.sort_values('total_revenue', ascending=False).copy()
    categorias_sorted['percentual_acumulado'] = categorias_sorted['percentual_faturamento'].cumsum()
    
    print("\nAnálise de Pareto (percentual acumulado do faturamento):")
    print(categorias_sorted[['category', 'percentual_faturamento', 'percentual_acumulado']].head(20))
    
    plt.figure(figsize=(16, 8))
    plt.bar(range(len(categorias_sorted)), categorias_sorted['percentual_faturamento'], color='skyblue')
    plt.plot(range(len(categorias_sorted)), categorias_sorted['percentual_acumulado'], color='red', marker='o')
    plt.title('Análise de Pareto - Faturamento por Categoria', fontsize=18)
    plt.xlabel('Categorias (ordenadas por faturamento)', fontsize=14)
    plt.ylabel('Percentual do Faturamento (%)', fontsize=14)
    plt.axhline(y=80, color='green', linestyle='--', label='Limite de 80%')
    plt.legend()
    plt.tight_layout()
    plt.show()
    
    n_categorias_80 = (categorias_sorted['percentual_acumulado'] <= 80).sum()
    print(f"\nNúmero de categorias que representam 80% do faturamento: {n_categorias_80}")
    print(f"Total de categorias: {len(categorias_sorted)}")
    print(f"Percentual de categorias que representam 80% do faturamento: {(n_categorias_80/len(categorias_sorted))*100:.2f}%")
    
    return categorias

# Parte E: Estados brasileiros com maior valor médio de pedido

In [19]:
def analise_estados_valor_pedido(conn):
    print_section("E) Estados brasileiros com maior valor médio de pedido")
    
    query = """
    SELECT 
        c.customer_state as estado,
        COUNT(DISTINCT fs.order_id) as num_pedidos,
        SUM(fs.price) as faturamento_total,
        SUM(fs.price) / COUNT(DISTINCT fs.order_id) as valor_medio_pedido
    FROM fact_sales fs
    JOIN dim_customers c ON fs.customer_id = c.customer_id
    GROUP BY c.customer_state
    ORDER BY valor_medio_pedido DESC
    """
    
    estados = run_query(query, conn)
    
    print("Valor médio de pedido por estado brasileiro:")
    print(estados)
    
    plt.figure(figsize=(16, 10))
    sns.barplot(x='valor_medio_pedido', y='estado', data=estados, palette='viridis')
    plt.title('Valor Médio de Pedido por Estado', fontsize=18)
    plt.xlabel('Valor Médio de Pedido (R$)', fontsize=14)
    plt.ylabel('Estado', fontsize=14)
    plt.tight_layout()
    plt.show()
    
    plt.figure(figsize=(16, 10))
    sns.barplot(x='faturamento_total', y='estado', data=estados.sort_values('faturamento_total', ascending=False), 
               palette='viridis')
    plt.title('Faturamento Total por Estado', fontsize=18)
    plt.xlabel('Faturamento Total (R$)', fontsize=14)
    plt.ylabel('Estado', fontsize=14)
    plt.tight_layout()
    plt.show()
    
    plt.figure(figsize=(16, 10))
    sns.barplot(x='num_pedidos', y='estado', data=estados.sort_values('num_pedidos', ascending=False), 
               palette='viridis')
    plt.title('Número de Pedidos por Estado', fontsize=18)
    plt.xlabel('Número de Pedidos', fontsize=14)
    plt.ylabel('Estado', fontsize=14)
    plt.tight_layout()
    plt.show()
    
    plt.figure(figsize=(16, 10))
    plt.scatter(estados['num_pedidos'], estados['valor_medio_pedido'], s=100, alpha=0.7)
    
    for i, row in estados.iterrows():
        plt.annotate(row['estado'], (row['num_pedidos'], row['valor_medio_pedido']), 
                     fontsize=12, ha='center')
    
    plt.title('Relação entre Número de Pedidos e Valor Médio por Estado', fontsize=18)
    plt.xlabel('Número de Pedidos', fontsize=14)
    plt.ylabel('Valor Médio de Pedido (R$)', fontsize=14)
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()
    
    return estados

In [ ]:
def main():
    """Função principal para executar todas as análises."""
    try:
        print_section("ANÁLISE EXPLORATÓRIA DE DADOS - E-COMMERCE BRASILEIRO")
        
        conn = sqlite3.connect(GOLD_DB_PATH)
        
        vendas_mensais = analise_volume_pedidos(conn)
        tempo_entrega = analise_tempo_entrega(conn)
        dados_frete = analise_frete_distancia(conn)
        categorias = analise_categorias_produtos(conn)
        estados = analise_estados_valor_pedido(conn)
        
        conn.close()
        
        print_section("ANÁLISE EXPLORATÓRIA CONCLUÍDA COM SUCESSO")
        
    except Exception as e:
        print(f"Erro durante a análise: {str(e)}")

if __name__ == "__main__":
    main()